In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
from matplotlib import pyplot as plt
import folium

In [3]:
nodes_NYC = gpd.read_file('data/nodes_NYC.geojson')

# check first five rows
nodes_NYC.head()

,x,y,Distance,geometry
0,-73.962176,40.792024,1458.626953,POINT (-73.96218 40.79202)
1,-73.963576,40.790720,1275.385010,POINT (-73.96358 40.79072)
2,-73.964181,40.788475,1075.475952,POINT (-73.96418 40.78848)
3,-73.966988,40.785589,744.629028,POINT (-73.96699 40.78559)
4,-73.973425,40.774267,1233.833008,POINT (-73.97343 40.77427)


In [4]:
nodes_NYC_under2000 =  nodes_NYC.loc[(nodes_NYC["Distance"] < 2000)]
nodes_NYC_halfmile =  nodes_NYC.loc[(nodes_NYC["Distance"] < 804.672)]

In [5]:
merged_NYC_tracts_SVI = gpd.read_file('data/merged_NYC_tracts_SVI.geojson')
merged_NYC_tracts_SVI = merged_NYC_tracts_SVI.to_crs(epsg=4326)

In [6]:
hurricane_evac_zones = gpd.read_file('data/Hurricane_Evacuation_Zones.geojson')
hurricane_evac_zones = hurricane_evac_zones.to_crs(epsg=4326)

hurricane_evac_zones = hurricane_evac_zones.rename(columns={"hurricane": "Zone"})
hurricane_evac_zones = hurricane_evac_zones.loc[(hurricane_evac_zones["Zone"] != "X") & (hurricane_evac_zones["Zone"] != "7")]

In [7]:
hurricane_evac_zones.head()

,Zone,shape_area,shape_leng,geometry
0,1,1331438777.89,2855052.33819,"MULTIPOLYGON (((-74.21178 40.50819, -74.21179 ..."
2,2,851138147.323,1787662.0299,"MULTIPOLYGON (((-74.23843 40.50268, -74.23833 ..."
3,3,390838625.801,1448569.7833,"MULTIPOLYGON (((-74.22929 40.50400, -74.23055 ..."
4,4,484377534.999,1599416.25916,"MULTIPOLYGON (((-74.24885 40.50074, -74.24961 ..."
5,6,541629414.584,2007990.98836,"MULTIPOLYGON (((-74.23826 40.50402, -74.23934 ..."


In [8]:
# just zone 1
hurricane_evac_zone1 = hurricane_evac_zones.loc[(hurricane_evac_zones["Zone"] == "1")]

In [11]:
# confirm
hurricane_evac_zone1

,Zone,shape_area,shape_leng,geometry
0,1,1331438777.89,2855052.33819,"MULTIPOLYGON (((-74.21178 40.50819, -74.21179 ..."


In [13]:
# define master aesthetic functions
def get_tract_style(feature):
    """Return a style dict."""
    return {"weight": 2, "color": "white"}


def get_zones_style(feature):
    """Return a style dict."""
    return {"weight": 1, "color": "red", "fillOpacity": 0.3}


def get_highlighted_style(feature):
    """Return a style dict when highlighting a feature."""
    return {"weight": 2, "color": "grey"}

In [16]:
import matplotlib.colors as mcolors
# use a reversed viridis color scheme for consistency
cmap = plt.get_cmap('viridis_r')

In [17]:
# define style function for SVI Overall
def get_style1(feature):
    # get the data value from the feature
    value = feature['properties']['SVI_Overall']
    
    # evaluate the color map
    # NOTE: value must between 0 and 1
    rgb_color = cmap(value) # this is a RGB tuple
    
    # convert to hex string
    color = mcolors.rgb2hex(rgb_color)
    
    # return the style dictionary
    return {'weight': 0.25, 'color': color, 'fillColor': color, "fillOpacity": 0.5}

In [18]:
# Normalize pop density column - very skewed
x = merged_NYC_tracts_SVI['Population_Density']
merged_NYC_tracts_SVI['Population_Density_normalized'] = (x - x.min()) / (x.max() - x.min())

In [20]:
# Define style function for Population Density

def get_style2(feature):
    # get the data value from the feature
    value = feature['properties']['Population_Density_normalized']
    
    # evaluate the color map
    # NOTE: value must between 0 and 1
    rgb_color = cmap(value) # this is a RGB tuple
    
    # convert to hex string
    color = mcolors.rgb2hex(rgb_color)
    
    # return the style dictionary
    return {'weight': 0.25, 'color': color, 'fillColor': color, "fillOpacity": 0.5}

In [21]:
# Convert to json
nodes_NYC_halfmile_json = nodes_NYC_halfmile.to_json()
merged_NYC_tracts_SVI_json = merged_NYC_tracts_SVI.to_json()
hurricane_evac_zones_json = hurricane_evac_zones.to_json()
hurricane_evac_zone1_json = hurricane_evac_zone1.to_json()

In [22]:
# Define style for hurricane evac zone 1
style3 = {'fillColor': '#D22B2B', 'color': '#D22B2B', 'weight': 0, 'fillOpacity':0.9}    

In [24]:
# MAP
map_master = folium.Map(
    location=[40.71, -74.00],
    tiles=None,
    zoom_start=11.25
)

base_map = folium.FeatureGroup(name='Basemap', overlay=True, control=False)
folium.TileLayer(tiles='CartoDB positron').add_to(base_map)
base_map.add_to(map_master)

# add the GeoJson to the map
folium.GeoJson(
    merged_NYC_tracts_SVI_json,
    name='Overall Social Vulnerability Index',
    style_function=get_style1,
    highlight_function=get_highlighted_style,
    overlay=False,
    show=True,
    tooltip=folium.GeoJsonTooltip(['Tract_Name_Full', 'SVI_Overall'])
).add_to(map_master)

# add the GeoJson to the map
folium.GeoJson(
    merged_NYC_tracts_SVI_json,
    name='Population Density (Normalized)',
    style_function=get_style2,
    highlight_function=get_highlighted_style,
    overlay = False,
    tooltip=folium.GeoJsonTooltip(['Tract_Name_Full', 'Population_Density_normalized'])
).add_to(map_master)

# add the GeoJson to the map - HURRICANE ZONES AS PROXY FOR STORM SURGE
folium.GeoJson(
    hurricane_evac_zone1_json,
    name='Hurricane Evacuation Zone',
    style_function = lambda x: style3,
    highlight_function=get_highlighted_style,
    tooltip=folium.GeoJsonTooltip(['Zone']),
    overlay = True
).add_to(map_master)


folium.GeoJson(
    nodes_NYC_halfmile_json,
    name="Accessible Nodes - 1/2 Mile",
    marker = folium.CircleMarker(radius = 2,
                                 weight = 0, #outline weight
                                 fill_color = 'black', 
                                 fill_opacity = 0.4),
    highlight_function=get_highlighted_style,
    tooltip=folium.GeoJsonTooltip(['Distance']),
).add_to(map_master)

folium.LayerControl().add_to(map_master)



# avoid a rendering bug by saving as HTML and re-loading
map_master.save('folium_map_final.html')

In [25]:
from IPython.display import IFrame # loads HTML files
filepath = 'folium_map_final.html'
map_master.save(filepath)
IFrame(filepath, width=800, height=600)